In [ ]:
import re
import nltk
import spacy

import pandas as pd

import matplotlib.pyplot as plt

from wordcloud import WordCloud
from unidecode import unidecode
from string import punctuation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk import word_tokenize
from nltk.corpus import stopwords

from spacy.lang.pt.stop_words import STOP_WORDS


%matplotlib inline

# Introdução

Um dos problemas do NLP está na qualidade dos dados, principalmente quando se trabalha com textos não formais (como redes sociais). Além disso a quantidade de material replicável em português é escasso, o que atrapalha bastante na modelagem. Sendo assim, o objetivo deste notebook é criar ferramentas para o pré-processamento dos textos.

# Funções para Pré-Processamento

## Limpeza do texto

A primeira função, e talvez a mais importante, é para limpar o texto. Nesta etapa é tratado tudo o que pode atrapalhar no processo de modelagem. Além disso, acredito que ela possa ser utilizada em qualquer situação quando se trata de NLP.

Na função abaixo são realizados os seguintes passos para limpeza do texto:

- Colocar o texto em caixa baixa (letras minúsculas);
- Excluir citações - útil para textos retirados de redes sociais;
- Excluir acentuação das palavras;
- Excluir html tags para textos retirados de fóruns e afins;
- Excluir números;
- Excluir URL's;
- Excluir pontuação e caracteres especiais.

Existem outras tratativas que podem ser adicionadas, mas algumas possuem restrições em questão de instalação (só funcionar no linux por exemplo) ou só ter disponível em inglês. Um exemplo é a função Speller da biblioteca autocorrect que corrige erros de digitação (com um erro logicamente) e só funciona em inglês.

In [ ]:
def limpar_texto(text):
    
    # Colocando todas as letras do texto em caixa baixa:
    text = text.lower()
    # Excluindo citações com @:
    text = re.sub('@[^\s]+', '', text)
    # Excluindo acentuação das palavras:
    text = unidecode(text)
    # Excluindo html tags, como <strong></strong>:
    text = re.sub('<[^<]+?>','', text)
    # Excluindo os números:
    text = ''.join(c for c in text if not c.isdigit())
    # Excluindo URL's:
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))', '', text)
    # Excluindo pontuação:
    text = ''.join(c for c in text if c not in punctuation)
    
    # Retornando o texto tratado tokenizado:
    
    return word_tokenize(text)

Testando a função:

In [ ]:
# O texto abaixo contém todas as situações para que seja feita a limpeza:

texto = """
<strong>Olá</strong> @usuario, vamos testar a função #clean_text?
Caso tenha dúvidas, uma boa pesquisa no www.google.com pode ajudar!
Mesmo que você tenha que pesquisar 100 vezes!
"""

texto_limpo = limpar_texto(texto)
print(texto_limpo)

## Remoção das Palavras de Parada (Stop Words)

As palavras de parada são aquelas que, dependendo do caso, podem ser consideradas irrelevantes para o conjunto de resultados a ser exibido em uma busca realizada em uma *search engine*. Por exemplo: o, para, com, foi.

**Quando removê-las de um texto?**

Quando se remove as palavras de parada geralmente o texto perde o seu contexto (ou sentido), o que pode atrapalhar alguns algoritmos de descoberta, principalmente aqueles que trabalham com redes neurais. Então é importante ter atenção.

Agora, quando montamos um **saco de palavras** (Bag-of-Words) as palavras de maior frequência provavelmente serão palavras de parada. Portanto, neste caso, removê-las é uma boa prática.

**Diferenças de linguagens e bibliotecas**

Cada linguagem possui as suas palavras de parada e a qualidade da remoção depende de vários fatores, dentre eles o quão correto está escrito o texto. Então, dependendo de onde ele foi coletado (redes sociais por exemplo), poderão aparecer ruídos após a remoção.
As duas principais bibliotecas que possuem funções para remoção de palavras de parada são spacy e nltk. Além disso, é possível (e recomendado pelo menos considerar) criar a própria lista de palavras para remover do texto nesta etapa.

In [ ]:
# Removendo as stopwords utilizando a lista do nltk e do spacy:

sw = list(set(stopwords.words('portuguese') + list(STOP_WORDS)))

def remove_stop_words(texts, stopwords = sw):
      
    new_texts = list()
    
    for word in texts:
        if word not in stopwords:
            new_texts.append(''.join(word))

    return new_texts


Vamos ver a nossa lista de palavras de parada:

In [ ]:
print(sw)

Agora vamos aplicá-la no texto de saída da função de limpeza:

In [ ]:
texto_sem_stop_words = remove_stop_words(texto_limpo)
print(texto_sem_stop_words)

Se estivéssemos criando um modelo a palavra 'cleantext' talvez não fosse importante. Vamos adicioná-la na nossa lista de palavras de palavra rodar novamente a função:

In [ ]:
texto_sem_stop_words = remove_stop_words(texto_limpo, sw + ['cleantext'])
print(texto_sem_stop_words)

## Lematização e stemização

Lematização e stemização são processos que reduzem as palavras. A vantagem de utilizá-los é a redução do vocabulário e abstração de significado. No caso do texto que estamos usando como exemplo, as palavras 'pesquisa' e 'pesquisar' poderiam ser reduzidas, pois em questão de significado elas agregam de forma igual no processo de modelagem.

- A lematização reduz a palavra ao seu lema, que é a forma no masculino e singular. No caso de verbos o lema é o infinitivo. Por exemplo, as palavras "menino", "meninos", "menininhos" são todas formas do lema: "menino".

- A stemização reduz a palavra ao seu radical. Por exemplo, as palavras "menino", "meninos", "menininhos" são todas formas do lema: "menin".

Um dos problemas de utilizar estas funções, principalmente a lematização, é que ele pode retornar alguns resultados estranhos, podendo perder o contexto do original. Mas mesmo assim o contexto ficará melhor do que extrair o radical.

Em termos de velocidade de processamento, extrair os radicais vai fazer com que mais palavras se agrupem e, consequentemente, uma possível modelagem fique mais rápida. Mas não quer dizer que a qualidade da predição vai ficar boa.

In [ ]:
# primeiramente é necessário realizar a instalação abaixo:

!python -m spacy download pt

In [ ]:
# Vamos criar uma função que mostra o texto original, a interpretação - da função - semântica dela e o lema

nlp = spacy.load("pt")

def verificar_lemma(words):
    
    text = ""
    pos = ""
    lemma = ""
    for word in nlp(words):
        text += word.text + "\t"
        pos += word.pos_ + "\t"
        lemma += word.lemma_ + "\t"

    print(text)
    print(pos)
    print(lemma)

Vamos verificar como ficaria a extração do lema em algumas frases:

In [ ]:
verificar_lemma('o sentido desta frase está errado')

Neste caso a função extraiu erroneamente o lema da palavra **sentido**, pois ela é um substantivo neste caso e o lema seria sentido. Além disso, separou a palavra desta em **d** e **esta**, sendo que o lema seria **deste**. Para as demais palavras a extração do lema funcionou corretamente.

In [ ]:
verificar_lemma('você está se sentindo bem?')

Neste caso a função extraiu o lema corretamente. O problema é que em um dataset com as duas frases, a palavra sentido seria agrupada, sendo que o lema das duas não é o mesmo, o que poderia acarretar em problemas em um algoritmo de aprendizagem. Agora vamos ver os radicais.

In [ ]:
# Vamos criar uma função que mostra o texto original e o stem de cada palavra

def verificar_radical(words):
    
    stemmer = nltk.stem.SnowballStemmer('portuguese')
    text = ""
    stem = ""
    
    for word in word_tokenize(words):

        text += word + "\t"
        stem += stemmer.stem(word) + "\t"
    
    print(text)
    print(stem)

Vamos verificar a extração do radical com as mesmas frases que utilizamos na função de lema:

In [ ]:
verificar_radical('o sentido desta frase esta errado')

In [ ]:
verificar_radical('você está se sentindo bem?')

Novamente a palavra **sentido** seria agrupada, mas neste caso, como é o radical, está correto.

Para concluir, utilizar ou não as funções de lema e radical é de escolha de quem está desenvolvendo. Caso seja escolhido por utilizar, aconselho dar uma atenção na qualidade das extrações.

## Nuvem de palavras

Uma nuvem de palavras é uma boa maneira de verificar quais palavras são mais ou menos relevantes em um dataset. Trata-se de um gráfico que *plota* as palavras indicando através de seu tamanho a quantidade de vezes que apareceu em todos os textos. Outra utilidade é adicionar novas palavras na lista de stop words que por ventura podem aparecer como relevante e não estarem na lista.

Abaixo temos uma função que cria uma nuvem de palavras:

In [ ]:
def nuvem_palavras(textos):
    
    # Juntando todos os textos na mesma string
    todas_palavras = ' '.join([texto for texto in textos])
    # Gerando a nuvem de palavras
    nuvem_palvras = WordCloud(width= 800, height= 500,
                              max_font_size = 110,
                              collocations = False).generate(todas_palavras)
    # Plotando nuvem de palavras
    plt.figure(figsize=(24,12))
    plt.imshow(nuvem_palvras, interpolation='bilinear')
    plt.axis("off")
    plt.show()

Vamos testar as funções de nuvem de palavras, CountVectorizer e TfidfVectorizer em um dataset de reviews traduzidos do imdb:

In [ ]:
df = pd.read_csv('../input/imdb-ptbr/imdb-reviews-pt-br.csv', nrows=1000)

In [ ]:
# vamos ver as primeiras cinco linhas do dataset:
df.head(5)

In [ ]:
# Construindo a nuvem de palavras:
nuvem_palavras(df["text_pt"])

Podemos observar que aparecem várias palavras de parada como relevantes no dataset. Portanto seria interessante removê-las, talvez adicionando a palavra "filme", pois se trata de *reviews* de filmes.

## CountVectorizer e TfidfVectorizer

Após a realização da limpeza dos textos é necessário transformar o dataset número para aplicar algum algoritmo de aprendizagem. Para tal, vamos apresentar duas técnicas: **CountVectorizer** e **TfidfVectorizer**.

O CountVectorizer agrupa todas as palavras e faz uma contagem da frequência de cada uma.

O TfidfVectorizer é um índice no qual o valor aumenta proporcionalmente à contagem das palavras, mas é compensado pela frequência da palavra no corpus (conjunto de todas as palavras do dataset). Este é o IDF, que significa *inverse document frequency part* - parte inversa da frequência no documento. A vantagem de usá-lo está no fato de que ele vai dar menos importância para palavras como as *stopwords*.

Abaixo temos funções para as duas técnicas:

In [ ]:
def countvectorizer(textos):

    vect = CountVectorizer()
    text_vect = vect.fit_transform(textos)
    
    return text_vect

def tfidfvectorizer(textos):
    
    vect = TfidfVectorizer(max_features=50)
    text_vect = vect.fit_transform(textos)
    
    return text_vect

Neste caso não iremos aplicá-las, pois não estamos interessados neste notebook em avançar para a etapa de criação de modelos, mas sim em preparar um dataset limpo para tal. Ao invés disso, vamos juntar todas as funções em uma classe e aplicá-las ao dataset do imdb.

# Classe de Pré-processamento para NLP

In [ ]:
class preprocess_nlp(object):
    
    def __init__(self, texts, stopwords = True, lemma=False, stem=False, wordcloud=True, numeric='tfidf'):
        
        self.texts = texts
        self.stopwords = stopwords
        self.lemma = lemma
        self.stem = stem
        self.wordcloud = wordcloud
        self.numeric = numeric
        self.new_texts = None
        self.stopwords_list = list()
        
    def clean_text(self):

        new_texts = list()

        for text in self.texts:

            text = text.lower()
            text = re.sub('@[^\s]+', '', text)
            text = unidecode(text)
            text = re.sub('<[^<]+?>','', text)
            text = ''.join(c for c in text if not c.isdigit())
            text = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))', '', text)
            text = ''.join(c for c in text if c not in punctuation)
            new_texts.append(text)
        
        self.new_texts = new_texts

    def create_stopwords(self):
        
        stop_words = list(set(stopwords.words('portuguese') + list(STOP_WORDS)))
        
        for word in stop_words:

            self.stopwords_list.append(unidecode(word))
       
    
    def add_stopword(self, word):
        
        self.stopwords_list += [word]
        

    def remove_stopwords(self):

        new_texts = list()

        for text in self.new_texts:

            new_text = ''

            for word in word_tokenize(text):

                if word.lower() not in self.stopwords_list:

                    new_text += ' ' + word

            new_texts.append(new_text)

        self.new_texts = new_texts


    def extract_lemma(self):
        
        nlp = spacy.load("pt")
        new_texts = list()

        for text in self.texts:

            new_text = ''

            for word in nlp(text):

                new_text += ' ' + word.lemma_

            new_texts.append(new_text)
        
        self.new_texts = new_texts
    

    def extract_stem(self):

        stemmer = nltk.stem.SnowballStemmer('portuguese')
        new_texts = list()

        for text in self.texts:

            new_text = ''

            for word in word_tokenize(text):

                new_text += ' ' + stemmer.stem(word)

            new_texts.append(new_text)

        self.new_texts = new_texts
    

    def word_cloud(self):

        all_words = ' '.join([text for text in self.new_texts])
        word_cloud = WordCloud(width= 800, height= 500,
                               max_font_size = 110,
                               collocations = False).generate(all_words)
        plt.figure(figsize=(24,12))
        plt.imshow(word_cloud, interpolation='bilinear')
        plt.axis("off")
        plt.show()
        

    def countvectorizer(self):

        vect = CountVectorizer()
        text_vect = vect.fit_transform(self.new_texts)

        return text_vect
    

    def tfidfvectorizer(self):

        vect = TfidfVectorizer(max_features=50)
        text_vect = vect.fit_transform(self.new_texts)

        return text_vect
    
    
    def preprocess(self):

        self.clean_text()
        
        if self.stopwords == True:
            self.create_stopwords()
            self.remove_stopwords()
            
        if self.lemma == True:
            self.extract_lemma()
        
        if self.stem == True:
            self.extract_stem() 
        
        if self.wordcloud == True:
            self.word_cloud()
        
        if self.numeric == 'tfidf':
            text_vect = self.tfidfvectorizer()
        elif self.numeric == 'count':
            text_vect = self.countvectorizer()
        else:
            print('metodo nao mapeado!')
            exit()
            
        return text_vect

Na criação desta classe coloquei alguns argumentos opcionais em relação à quais funções serão executadas no pré-processamento. O padrão será remover as palavras de parada, fazer a limpeza do texto, apresentar a nuvem de palavras e aplicar o TfidfVectorizer.

Agora vamos aplicar na base do imdb:

In [ ]:
prepro = preprocess_nlp(df['text_pt'], numeric='count')
#adicionando as palavras filme e filmes na lista de palavras de parada, pois elas são irrelevantes neste contexto
prepro.add_stopword('filme') 
prepro.add_stopword('filmes') 
sparse_matrix = prepro.preprocess()

# Conclusão

Neste documento abordamos algumas funções de pré-processamento de textos para análise de sentimento onde foi possível notar que existe uma gama enorme de possibilidades para tal. Ao final, criamos uma classe, sem aprofundar muito em cada função, para realizar a tratativa de um dataframe de textos com algumas opções e aplicamos em um dataset selecionado.
A partir daí é possível realizar estudos exploratórios e preditivos na base resultante.